## Keras with TensorFlow using Nvidia architect
### Generator version

In [13]:
# import required components
import os
import csv
import cv2
import numpy as np
import sklearn
from sklearn.utils import shuffle

In [14]:
# read the log file
samples=[]
with open("./data/driving_log.csv") as csvfile:
    reader=csv.reader(csvfile)
    next(reader)
    for line in reader:
        samples.append(line)
print(len(samples))

8036


In [15]:
from sklearn.model_selection import train_test_split
train_samples, validation_samples = train_test_split(samples, test_size=0.2)
print(len(train_samples),len(validation_samples))

6428 1608


In [16]:
# define the gererator function
def generator(samples, batch_size=32):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            measurements = []
            for batch_sample in batch_samples:
                for i in range(3):
                    name = './data/IMG/' + batch_sample[i].split('/')[-1]
                    image = cv2.imread(name)
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                    measurement = float(batch_sample[3])
                    if i==1:
                        measurement = measurement + 0.25
                    elif i==2:
                        measurement = measurement - 0.25
                    
                    images.append(image)
                    measurements.append(measurement)

            augmented_images, augmented_measurements = [], []
            for image, measurement in zip(images, measurements):
                    augmented_images.append(image)
                    augmented_measurements.append(measurement)
                    augmented_images.append(cv2.flip(image, 1))
                    augmented_measurements.append(measurement * -1.0)
                
            X_train = np.array(augmented_images)
            y_train = np.array(augmented_measurements)
            
            yield sklearn.utils.shuffle(X_train, y_train)

In [17]:
# compile and train the model using the generator function
train_generator = generator(train_samples, batch_size=32)
validation_generator = generator(validation_samples, batch_size=32)

In [18]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Dropout
from keras.layers.convolutional  import Convolution2D, MaxPooling2D, Cropping2D


model=Sequential()
model.add(Lambda(lambda x: (x/255.0) - 0.5,input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((70,25),(0,0))))
model.add(Convolution2D(24,5,5,subsample=(2,2),activation="relu"))
model.add(Dropout(0.2))
model.add(Convolution2D(36,5,5,subsample=(2,2),activation="relu"))
model.add(Dropout(0.2))
model.add(Convolution2D(48,5,5,subsample=(2,2),activation="relu"))
model.add(Dropout(0.2))
model.add(Convolution2D(64,3,3,activation="relu"))
model.add(Dropout(0.2))
model.add(Convolution2D(64,3,3,activation="relu"))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))

model.compile(loss='mse', optimizer ='adam')
model.fit_generator(train_generator, samples_per_epoch=len(train_samples)*6,
                    validation_data=validation_generator,nb_val_samples=len(validation_samples)*6, 
                    nb_epoch=10, verbose=1)

model.save('model.h5')
print("Model saved!")

Epoch 1/10
38568/38568 [==============================] - 81s - loss: 0.0266 - val_loss: 0.0231
Epoch 2/10
38568/38568 [==============================] - 80s - loss: 0.0219 - val_loss: 0.0225
Epoch 3/10
38568/38568 [==============================] - 80s - loss: 0.0208 - val_loss: 0.0221
Epoch 4/10
38568/38568 [==============================] - 80s - loss: 0.0202 - val_loss: 0.0213
Epoch 5/10
38568/38568 [==============================] - 80s - loss: 0.0196 - val_loss: 0.0211
Epoch 6/10
38568/38568 [==============================] - 80s - loss: 0.0190 - val_loss: 0.0206
Epoch 7/10
38568/38568 [==============================] - 80s - loss: 0.0186 - val_loss: 0.0199
Epoch 8/10
38568/38568 [==============================] - 80s - loss: 0.0180 - val_loss: 0.0197
Epoch 9/10
38568/38568 [==============================] - 80s - loss: 0.0177 - val_loss: 0.0192
Epoch 10/10
38568/38568 [==============================] - 80s - loss: 0.0174 - val_loss: 0.0191
Model saved!
